In [165]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product

In [209]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """

        :param categorical_features: list of str, the name of the categorical columns to encode
        : input parameter

        :param n_splits: the number of splits used in mean encoding
        : cross validation setting

        :param target_type: str, 'regression' or 'classification'
        :
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):

        X_train = pd.DataFrame( X_train[variable].copy())
        X_test = pd.DataFrame( X_test[variable].copy())
        
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
        
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
        #print 'col_avg_y'
        #print col_avg_y.head()
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)
        #print col_avg_y
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        # 下面是对测试数据采用对应的映射, 如果出现缺失值采用填充的方法
        #nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)} 
            # print self.target_values , self.categorical_features
            # self.target_values   { 0, 1 }
            # self.categorical_features  { 1,2,3,4}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                print nf_name
                X_new[nf_name] = np.nan
                # cv.split(y, y)
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable)[nf_name]
                    #X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                    #    nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable)[nf_name]
                    #X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new

In [210]:
df = pd.read_csv('test.csv')
df['device'] = df['device'].astype('category')
df['test'] = df['test'].astype('category')
df['is_attributed'] = df['is_attributed'].astype('category')
df.describe()

,test,device,is_attributed
count,50,50,50
unique,1,4,2
top,0,1,1
freq,50,24,25


In [211]:
#df['1_pred_2'] = np.nan

In [212]:
y = df['is_attributed']
x = df.drop(['is_attributed'], axis=1)
data = MeanEncoder(['device'])

In [213]:
x_new = data.fit_transform(x,y)

device_pred_0
device_pred_1


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:52: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [214]:
x_new = data.transform(x)

In [215]:
x_new

,test,device,device_pred_0,device_pred_1
0,0,1,0.500000,0.500000
1,0,3,0.557672,0.442328
2,0,1,0.500000,0.500000
3,0,4,0.498737,0.501263
4,0,3,0.557672,0.442328
5,0,1,0.500000,0.500000
6,0,2,0.461827,0.538173
7,0,1,0.500000,0.500000
8,0,2,0.461827,0.538173
9,0,1,0.500000,0.500000


In [ ]:
dic  = {'k': 3, 'f': 5}
a= MeanEncoder(['asdf'], prior_weight_func = dic)
#dict(a.prior_weight_func, np=np)
#print map(a.prior_weight_func, [1,100])
#print dict(dic , np=np)